# Простий класифікатор зображень

Цей ноутбук демонструє, як класифікувати зображення за допомогою попередньо навченої нейронної мережі.

**Що ви дізнаєтесь:**
- Як завантажити та використовувати попередньо навчену модель
- Попередня обробка зображень
- Виконання прогнозів на зображеннях
- Розуміння рівнів впевненості

**Випадок використання:** Визначення об'єктів на зображеннях (наприклад, "кіт", "собака", "автомобіль" тощо)

---


## Крок 1: Імпорт необхідних бібліотек

Давайте імпортуємо інструменти, які нам знадобляться. Не хвилюйтеся, якщо ви поки що не розумієте всі з них!


In [ ]:
# Core libraries
import numpy as np
from PIL import Image
import requests
from io import BytesIO

# TensorFlow for deep learning
try:
    import tensorflow as tf
    from tensorflow.keras.applications import MobileNetV2
    from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, decode_predictions
    print("✅ TensorFlow loaded successfully!")
    print(f"   Version: {tf.__version__}")
except ImportError:
    print("❌ Please install TensorFlow: pip install tensorflow")

## Крок 2: Завантаження попередньо навченої моделі

Ми використаємо **MobileNetV2**, нейронну мережу, яка вже навчена на мільйонах зображень.

Це називається **Трансферне навчання** - використання моделі, яку хтось інший вже навчив!


In [ ]:
print("📦 Loading pre-trained MobileNetV2 model...")
print("   This may take a minute on first run (downloading weights)...")

# Load the model
# include_top=True means we use the classification layer
# weights='imagenet' means it was trained on ImageNet dataset
model = MobileNetV2(weights='imagenet', include_top=True)

print("✅ Model loaded!")
print(f"   The model can recognize 1000 different object categories")

## Крок 3: Допоміжні функції

Давайте створимо функції для завантаження та підготовки зображень для нашої моделі.


In [ ]:
def load_image_from_url(url):
    """
    Load an image from a URL.
    
    Args:
        url: Web address of the image
        
    Returns:
        PIL Image object
    """
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    return img


def prepare_image(img):
    """
    Prepare an image for the model.
    
    Steps:
    1. Resize to 224x224 (model's expected size)
    2. Convert to array
    3. Add batch dimension
    4. Preprocess for MobileNetV2
    
    Args:
        img: PIL Image
        
    Returns:
        Preprocessed image array
    """
    # Resize to 224x224 pixels
    img = img.resize((224, 224))
    
    # Convert to numpy array
    img_array = np.array(img)
    
    # Add batch dimension (model expects multiple images)
    img_array = np.expand_dims(img_array, axis=0)
    
    # Preprocess for MobileNetV2
    img_array = preprocess_input(img_array)
    
    return img_array


def classify_image(img):
    """
    Classify an image and return top predictions.
    
    Args:
        img: PIL Image
        
    Returns:
        List of (class_name, confidence) tuples
    """
    # Prepare the image
    img_array = prepare_image(img)
    
    # Make prediction
    predictions = model.predict(img_array, verbose=0)
    
    # Decode predictions to human-readable labels
    # top=5 means we get the top 5 most likely classes
    decoded = decode_predictions(predictions, top=5)[0]
    
    # Convert to simpler format
    results = [(label, float(confidence)) for (_, label, confidence) in decoded]
    
    return results


print("✅ Helper functions ready!")

## Крок 4: Тестування на зразкових зображеннях

Спробуймо класифікувати кілька зображень з інтернету!


In [ ]:
# Sample images to classify
# These are from Unsplash (free stock photos)
test_images = [
    {
        "url": "https://images.unsplash.com/photo-1514888286974-6c03e2ca1dba?w=400",
        "description": "A cat"
    },
    {
        "url": "https://images.unsplash.com/photo-1552053831-71594a27632d?w=400",
        "description": "A dog"
    },
    {
        "url": "https://images.unsplash.com/photo-1511919884226-fd3cad34687c?w=400",
        "description": "A car"
    },
]

print(f"🧪 Testing on {len(test_images)} images...")
print("=" * 70)

### Класифікуйте кожне зображення


In [ ]:
for i, img_data in enumerate(test_images, 1):
    print(f"\n📸 Image {i}: {img_data['description']}")
    print("-" * 70)
    
    try:
        # Load image
        img = load_image_from_url(img_data['url'])
        
        # Display image
        display(img.resize((200, 200)))  # Show smaller version
        
        # Classify
        results = classify_image(img)
        
        # Show predictions
        print("\n🎯 Top 5 Predictions:")
        for rank, (label, confidence) in enumerate(results, 1):
            # Create a visual bar
            bar_length = int(confidence * 50)
            bar = "█" * bar_length
            
            print(f"  {rank}. {label:20s} {confidence*100:5.2f}% {bar}")
        
    except Exception as e:
        print(f"❌ Error: {e}")

print("\n" + "=" * 70)

## Крок 5: Спробуйте власні зображення!

Замініть URL нижче на будь-який URL зображення, яке ви хочете класифікувати.


In [ ]:
# Try your own image!
# Replace this URL with any image URL
custom_image_url = "https://images.unsplash.com/photo-1472491235688-bdc81a63246e?w=400"  # A flower

print("🖼️  Classifying your custom image...")
print("=" * 70)

try:
    # Load and show image
    img = load_image_from_url(custom_image_url)
    display(img.resize((300, 300)))
    
    # Classify
    results = classify_image(img)
    
    # Show results
    print("\n🎯 Top 5 Predictions:")
    print("-" * 70)
    for rank, (label, confidence) in enumerate(results, 1):
        bar_length = int(confidence * 50)
        bar = "█" * bar_length
        print(f"  {rank}. {label:20s} {confidence*100:5.2f}% {bar}")
    
    # Highlight top prediction
    top_label, top_confidence = results[0]
    print("\n" + "=" * 70)
    print(f"\n🏆 Best guess: {top_label} ({top_confidence*100:.2f}% confident)")
    
except Exception as e:
    print(f"❌ Error: {e}")
    print("   Make sure the URL points to a valid image!")

## 💡 Що щойно сталося?

1. **Ми завантажили попередньо навчену модель** - MobileNetV2 була навчена на мільйонах зображень  
2. **Ми попередньо обробили зображення** - Змінили розмір і форматували їх для моделі  
3. **Модель зробила прогнози** - Вона видала ймовірності для 1000 класів об'єктів  
4. **Ми декодували результати** - Перетворили числа на зрозумілі для людини назви  

### Розуміння рівнів впевненості

- **90-100%**: Дуже впевнено (майже напевно правильно)  
- **70-90%**: Впевнено (ймовірно правильно)  
- **50-70%**: Помірно впевнено (можливо правильно)  
- **Менше 50%**: Невпевнено (сумнівно)  

### Чому прогнози можуть бути неправильними?

- **Незвичайний кут або освітлення** - Модель була навчена на типових фотографіях  
- **Кілька об'єктів** - Модель очікує один основний об'єкт  
- **Рідкісні об'єкти** - Модель знає лише 1000 категорій  
- **Низька якість зображення** - Розмиті або пікселізовані зображення складніші для аналізу  

---


## 🚀 Наступні кроки

1. **Спробуйте різні зображення:**
   - Знайдіть зображення на [Unsplash](https://unsplash.com)
   - Клацніть правою кнопкою миші → "Копіювати адресу зображення", щоб отримати URL

2. **Експериментуйте:**
   - Що відбувається із абстрактним мистецтвом?
   - Чи може розпізнавати об'єкти з різних ракурсів?
   - Як система справляється з кількома об'єктами?

3. **Дізнайтеся більше:**
   - Досліджуйте [уроки з комп'ютерного зору](../lessons/4-ComputerVision/README.md)
   - Навчіться тренувати власний класифікатор зображень
   - Зрозумійте, як працюють CNN (Convolutional Neural Networks)

---

## 🎉 Вітаємо!

Ви щойно створили класифікатор зображень, використовуючи сучасну нейронну мережу!

Ця ж техніка використовується для:
- Google Photos (організація ваших фотографій)
- Автономних автомобілів (розпізнавання об'єктів)
- Медичної діагностики (аналіз рентгенівських знімків)
- Контролю якості (виявлення дефектів)

Продовжуйте досліджувати та навчатися! 🚀



---

**Відмова від відповідальності**:  
Цей документ був перекладений за допомогою сервісу автоматичного перекладу [Co-op Translator](https://github.com/Azure/co-op-translator). Хоча ми прагнемо до точності, будь ласка, зверніть увагу, що автоматичні переклади можуть містити помилки або неточності. Оригінальний документ на його рідній мові слід вважати авторитетним джерелом. Для критично важливої інформації рекомендується професійний людський переклад. Ми не несемо відповідальності за будь-які непорозуміння або неправильні тлумачення, що виникають внаслідок використання цього перекладу.
